[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab6/lab6-parte1.ipynb)
# Práctica 6: Redes neuronales convolucionales - Parte 1 - FF vs CNN


### Pre-requisitos. Instalar paquetes

Para la primera parte de este Laboratorio 6 necesitaremos TensorFlow y TensorFlow-Datasets. Además, como habitualmente, fijaremos la semilla aleatoria para asegurar la reproducibilidad de los experimentos.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

#Fijamos la semilla para poder reproducir los resultados
import os
import numpy as np
import random
seed=1234567
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

Además, cargamos también APIs que vamos a emplear para que el código quede más legible

In [ ]:
#API de Keras, modelo Sequential y la capa Dense 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense 
#Para mostrar gráficas
from matplotlib import pyplot

### Carga del conjunto de datos

En esta ocasión trabajaremos con el conjunto de imágenes *mnist*, que representa dígitos escritos a mano.

In [ ]:
import tensorflow_datasets as tfds

# El parámetro with_info=True nos permite acceder a información sobre el dataset
# Carga el conjunto de datos mnist. Usaremos el primer 80% de la partición train para ds_train y el 20% restante para ds_val. ds_test tomará la partición test.
(ds_train, ds_test, ds_val), ds_info = tfds.load(..., with_info=True, as_supervised=True)


# En dicha información se encuentran los nombres de las clases y las dimensiones de las imágenes
NUM_CLASSES = ds_info.features['label'].num_classes
nombres_clases = ds_info.features['label'].names
dimensiones = ds_info.features['image'].shape
print("Hay %d clases"%NUM_CLASSES)

# Para comprobar que se ha cargado tomamos un elemento y lo mostramos
ej_imagen, ej_etiqueta = next(iter(ds_train.take(1)))
pyplot.imshow(ej_imagen[:,:,0])
pyplot.xlabel(nombres_clases[ej_etiqueta.numpy()])
pyplot.show()

## Preprocesado de los datos

La etiqueta que nos suministra el dataset es numérica. Sin embargo, nosotros prediciremos un vector con tantas componentes como clases, donde cada componente estima la probabilidad de que el ejemplo pertenezca a una clase. Por tanto, hay que convertir la etiqueta suministrada a codificación one_hot con la función [tf.one_hot](https://www.tensorflow.org/api_docs/python/tf/one_hot)

Por otra parte, cada color de cada pixel de la imagen viene indicado con un entero entre 0 y 255. Para entrenar es preferible que se indiquen con números entre 0 y 1, por lo que deberemos escalar la imagen dividiendo su tensor por 255.

**PISTA: el número de clases se ha almacenado anteriormente en la variable NUM_CLASSES**

In [ ]:
dimensiones = ej_imagen.shape

## TODO: convierte las etiquetas a tipo one hot.
ds_train = ds_train.map(lambda image, label: (tf.cast(image,tf.float32)/255.0, ...))
ds_test = ...
ds_val = ...

## Ajustando los datos con un red neuronal feed-forward

Vamos a modelar los datos con una red feed-forward que tenga capas de 40, 25 y 16 unidades (todas con activación ReLU). Debemos tener en cuenta que las imágenes son tensores de dimensión 3 (su `shape` es (28,28,1)), mientras que la entrada de nuestras capas Dense debe ser un tensor de dimensión 1. Para adecuar la entrada a lo que necesitamos, vamos a "aplanar" los tensores de las imágenes, que pasarán de `shape` (28,28,1) a `shape` (784). Utilizaremos para ello una capa `Flatten`.

Por último, la salida de nuestro modelo debe tener tantas componentes como clases distintas tiene el conjunto. Como queremos que la salida aproxime la probabilidad de las distintas clases, lo habitual sería poner una función de activación *softmax*, pero en este caso, por razones de eficiencia del entrenamiento, es mejor dejar una salida lineal y posteriormente indicarle a la función de pérdida que las salidas vienen en ese formato.

In [ ]:
# TODO - Crea el modelo descrito
model = ...

#Construimos el modelo y mostramos 
model.build()
print(model.summary())

# VERIFICACIÓN
assert model.count_params()==33011, 'Revisa la arquitectura de tu modelo'

### Entrenamiento del modelo
Vamos a establecer la función de pérdida, el optimizador (Adam con el LR por defecto) y la métrica que nos servirá para evaluar el rendimiento del modelo entrenado (precisión categórica).

Como intentamos predecir una clase entre varias, nuestra función de pérdida debe ser la [entropía cruzada categórica](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy). Aquí es donde le indicaremos que la salida de nuestra red no son valores entre 0 y 1, sino que son valores reales que deben ser utilizados como *logits* por la función softmax.

In [ ]:
#TODO - Compila el modelo con los parámetros indicados
model.compile(...)

Como siempre, entrenaremos el modelo usando `model.fit`. Para ello, previamente debemos indicar a nuestro dataset que haga lotes de 128 elementos. Le indicaremos también que baraje los datos utilizando un buffer de 5 veces el tamaño de lote. La aleatorización debe hacerse antes de la partición en lotes, para que se aleatoricen los elementos y no los lotes.

In [ ]:
# TODO - Baraja y trocea los datasets en lotes.
ds_train_batch = ...
ds_val_batch = ...

# TODO - Entrena el modelo. Con 16 epochs será suficiente.
# Haz que nos ofrezca también las mediciones de pérdida y precisión sobre el conjunto de validación,
# para saber si el modelo está sobreajustando.
history = model.fit(...)

In [ ]:
# plot training history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.title('Loss')
pyplot.show()

# plot training history
pyplot.plot(history.history['categorical_accuracy'], label='train')
pyplot.plot(history.history['val_categorical_accuracy'], label='val')
pyplot.legend()
pyplot.title('Accuracy')
pyplot.show()

### Verificación del rendimiento
Aprovecharemos el conjunto de test para comprobar la capacidad de generalización de nuestro modelo.

In [ ]:
# TODO - Evalúa el modelo sobre el conjunto de test. Previamente deberás hacer lotes con el conjunto de test.
print("Evaluación sobre el conjunto TEST:")
ds_test_batch = ...
...

Si todo ha ido correctamente deberías haber obtenido un valor de precisión sobre el conjunto de test comparable a los obtenidos con los conjuntos de entrenamiento y validación, lo que indica que el modelo generaliza bien a otros datos del conjunto original pero... ¿tenemos un buen modelo?

Vamos a comprobar la robustez del modelo haciendo pequeños desplazamientos de las imágenes originales. Utilizaremos un pequeño modelo que aplique una traslación aleatoria de hasta un 10% del tamaño de la imagen a cada una de las imágenes de test. Nos ayudaremos de la capa `RandomTranslation` de preprocesado de Keras.

In [ ]:
from tensorflow.keras.layers import RandomTranslation

translator = Sequential(
                [RandomTranslation(height_factor=0.1, width_factor=0.1)]
            )

# TODO - Aplica la red translator a cada imagen
ds_test_desplazado = ds_test_batch.map(lambda image, label: (...,label))


# TODO - Toma un elemento de ds_test_desplazado y muéstralo con pyplot
ej_imagen_desplazada = next(iter(ds_test_desplazado.take(1)))[0][0]

pyplot.imshow(ej_imagen_desplazada[:,:,0])
pyplot.xlabel(nombres_clases[ej_etiqueta.numpy()])
pyplot.title('imagen desplazada')
pyplot.show()

Comprobemos ahora la precisión sobre este nuevo conjunto de imágenes que han sido ligeramente desplazadas.

In [ ]:
print("Evaluación sobre el conjunto TEST DESPLAZADO:")
...

Si todo ha ido bien, deberías haber comprobado que estas pequeñas traslaciones son suficientes para que la precisión del modelo baje sustancialmente. Las redes feed-forward no son robustas ante este tipo de perturbaciones.

## Comparativa con una red convolucional

Declara ahora un modelo convolucional con la siguiente arquitectura:
 1. [Convolución 2D](https://keras.io/api/layers/convolution_layers/convolution2d/) de 8 filtros y tamaño de kernel 3, con activación ReLU
 1. [Pooling 2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/) tomando el máximo de cada grupo de 2x2
 1. [Convolución 2D](https://keras.io/api/layers/convolution_layers/convolution2d/) de 8 filtros y tamaño de kernel 3, con activación ReLU
 1. [Pooling 2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/) tomando el máximo de cada grupo de 2x2
 1. Capa Densa (requiere aplanado previo) de 32 unidades y activación ReLU
 
Ejecuta la siguiente celda y repite el compilado, entrenamiento y verificaciones posteriores para observar la diferencia.

In [ ]:
# TODO
model = ...

#Construimos el modelo y mostramos 
model.build()
print(model.summary())

# VERIFICACIÓN
assert model.count_params()==7426, 'Revisa la arquitectura de tu modelo'

### Reflexiones sobre la comparativa
 - ¿Qué has observado en el rendimiento?
 - ¿Cuántos parámetros tiene la red convolucional respecto a la *feed-forward*
 - ¿Cómo ha cambiado el tiempo de ejecución?
 - ¿Es más robusta frente a los desplazamientos esta red?